In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Mobile sensor data acquisition example
#------------------------------------------------------------------------------------------------------------------
import time
import requests
import numpy as np
import threading

import random
import pickle
from datetime import datetime

from scipy.interpolate import interp1d

conditions = [('Walking', 0), ('Running', 1), ('Jumping up and down', 2),
              ('Greeting', 3), ('Sit down', 4), ('Shadow Boxing', 5)]

n_trials = 8      # Menos pruebas por actividad
n_windows = 5     # Menos ventanas por prueba
window_time = 2   # Igual duración de ventana
sampling_rate = 50  # Aumenta frecuencia (más datos por segundo)            # Number of windows for each trial

fixation_cross_time = 2     # Time in seconds for attention fixation
preparation_time = 2        # Time in seconds for preparation before each trial
rest_time = 2               # Time in seconds for rest between trials
        # Sampling rate in Hz of the output data
max_samp_rate = 5000        # Maximum possible sampling rate
max_window_samples = int(window_time*max_samp_rate)     # Maximum number of samples in each window

trials = []
for _ in range(n_trials):
    trials.extend(conditions)
random.shuffle(trials)

trial_time = fixation_cross_time + preparation_time + n_windows*window_time + rest_time

# Communication parameters
IP_ADDRESS = '10.43.37.93'  # Replace with your device's IP address
COMMAND = 'accX&accY&accZ&acc_time&gyroX&gyroY&gyroZ'
BASE_URL = f"http://{IP_ADDRESS}/get?{COMMAND}"

# Data buffer
n_signals = 6   # accX, accY, accZ, gyroX, gyroY, gyroZ
buffer_size = int(2 * len(trials)*trial_time * max_samp_rate)       # Buffer size  (2 times the number of samples in the complete experiment)

buffer = np.zeros((buffer_size, n_signals + 1), dtype='float64')    # Buffer for storing data (channel 0 is time)
buffer_index = 0                                                    # Index for the next data point to be written

# Flag for stopping the data acquisition
stop_recording_flag = threading.Event()

# Mutex for thread-safe access to the buffer
buffer_lock = threading.Lock()

# Function for continuously fetching data from the mobile device
def fetch_data():    
    sleep_time = 1. / max_samp_rate 
    while not stop_recording_flag.is_set():
        try:
            response = requests.get(BASE_URL, timeout=0.5)
            response.raise_for_status()            
            data = response.json()

            global buffer, buffer_index    
            
            with buffer_lock:  # Ensure thread-safe access to the buffer
                buffer[buffer_index:, 0] = data["buffer"]["acc_time"]["buffer"][0]
                buffer[buffer_index:, 1] = data["buffer"]["accX"]["buffer"][0]
                buffer[buffer_index:, 2] = data["buffer"]["accY"]["buffer"][0]
                buffer[buffer_index:, 3] = data["buffer"]["accZ"]["buffer"][0]
                buffer[buffer_index:, 4] = data["buffer"]["gyroX"]["buffer"][0]
                buffer[buffer_index:, 5] = data["buffer"]["gyroY"]["buffer"][0]
                buffer[buffer_index:, 6] = data["buffer"]["gyroZ"]["buffer"][0]

                buffer_index += 1
            
        except Exception as e:
            print(f"Error fetching data: {e}")

        time.sleep(sleep_time)

# Function for stopping the data acquisition
def stop_recording():
    stop_recording_flag.set()
    recording_thread.join()
    
# Start data acquisition
recording_thread = threading.Thread(target=fetch_data, daemon=True)
recording_thread.start()

# Run experiment
print ("********* Experiment in progress *********")    
time.sleep(fixation_cross_time)  

window_info = []
count  = 0
for t in trials:

    # Fixation cross    
    count = count + 1;
    print ("\n********* Trial {}/{} *********".format(count, len(trials)))    
    time.sleep(fixation_cross_time)    
    
    # Preparation time
    print (t[0])
    time.sleep(preparation_time)

    # Task
    for window in range(n_windows):                
        time.sleep(window_time)
        #with buffer_lock:  # Ensure thread-safe access to the buffer
        window_info.append((t[0], t[1], buffer_index))  

    # Rest time    
    print ("----Rest----")
    time.sleep(rest_time)

# Stop data acquisition
stop_recording()

# Calculate average sampling rate
t = buffer[:buffer_index, 0]    # Time data
diff_t = np.diff(t)             # Time differences

print("Min sampling rate: {:.2f} Hz".format(1. / np.max(diff_t)))
print("Max sampling rate: {:.2f} Hz".format(1. / np.min(diff_t)))
print("Average sampling rate: {:.2f} Hz".format(1. / np.mean(diff_t)))

# Interpolation functions for uniform sampling
interp_x1 = interp1d(t, buffer[:buffer_index, 1], kind='linear', fill_value="extrapolate")
interp_x2 = interp1d(t, buffer[:buffer_index, 2], kind='linear', fill_value="extrapolate")
interp_x3 = interp1d(t, buffer[:buffer_index, 3], kind='linear', fill_value="extrapolate")
interp_g1 = interp1d(t, buffer[:buffer_index, 4], kind='linear', fill_value="extrapolate")
interp_g2 = interp1d(t, buffer[:buffer_index, 5], kind='linear', fill_value="extrapolate")
interp_g3 = interp1d(t, buffer[:buffer_index, 6], kind='linear', fill_value="extrapolate")

# Separate the data for each trial
window_samples = int(sampling_rate * window_time)  # Number of samples in each window
data = []
for w in window_info:
    condition = w[0]        # Condition name
    condition_id = w[1]     # Condition ID
    start_index =  w[2]     # Start index of the window in the buffer

    # Calculate the uniform time vector for the window
    t_start = buffer[start_index, 0]    # Start time of the window
    t_uniform = np.linspace(t_start, t_start + window_time, int(window_time * sampling_rate))    

    # Interpolate the signals for the uniform time vector
    signal_data = np.column_stack((
    interp_x1(t_uniform), interp_x2(t_uniform), interp_x3(t_uniform),
    interp_g1(t_uniform), interp_g2(t_uniform), interp_g3(t_uniform)
))
    
    # Append the data for this window
    data.append((condition, condition_id, signal_data))


nombre = "vertiz_d"  
outputFile = open(f"{nombre}_data.obj", 'wb')
pickle.dump(data, outputFile)
outputFile.close()

#------------------------------------------------------------------------------------------------------------------
#   End of file
#------------------------------------------------------------------------------------------------------------------

In [ ]:
import pickle

# File names to combine
archivos = ['daniel_sancho_data.obj','gustavo_data.obj', 'vertiz_d_data.obj']

# List to hold all data
all_data = []

for archivo in archivos:
    with open(archivo, 'rb') as f:
        data = pickle.load(f)
        all_data.extend(data)

with open('data_combined.obj', 'wb') as f:
    pickle.dump(all_data, f)

print("Combined data saved to 'datos_combined_new.obj'.")

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Mobile sensor data acquisition and processing
#------------------------------------------------------------------------------------------------------------------
import pickle
import numpy as np
from scipy import stats

# Load the combined data file
file_name = 'data_combined.obj'
with open(file_name, 'rb') as inputFile:
    experiment_data = pickle.load(inputFile)

# Helper function: calculate signal energy
def energy(sig):
    return np.sum(sig ** 2) / len(sig)

# Helper function: count zero crossings in the signal
def zero_crossings(sig):
    return ((sig[:-1] * sig[1:]) < 0).sum()

# List to store feature vectors
features = []

# Process each trial
for tr in experiment_data:
    label = tr[1]       # Activity ID (e.g., 0, 1, ..., 5)
    signal = tr[2]      # Signal shape: (samples, 6 axes: accX, accY, accZ, gyroX, gyroY, gyroZ)

    feat = [label]      # Start the feature list with the activity label

    for i in range(6):  # Loop through each axis: accX, accY, accZ, gyroX, gyroY, gyroZ
        sig = signal[:, i]

        feat.append(np.mean(sig))                # Mean
        feat.append(np.std(sig))                 # Standard deviation
        feat.append(np.max(sig))                 # Maximum value
        feat.append(np.min(sig))                 # Minimum value
        feat.append(stats.skew(sig))             # Skewness
        feat.append(stats.kurtosis(sig))         # Kurtosis
        feat.append(energy(sig))                 # Energy
        feat.append(np.sqrt(np.mean(sig ** 2)))  # RMS
        feat.append(zero_crossings(sig))         # Zero-crossing count

    acc_mag = np.linalg.norm(signal[:, 0:3], axis=1)  # Magnitude of accX, accY, accZ
    gyro_mag = np.linalg.norm(signal[:, 3:6], axis=1)  # Magnitude of gyroX, gyroY, gyroZ

    feat.append(np.mean(acc_mag))        # Mean acc magnitude
    feat.append(np.std(acc_mag))         # Std acc magnitude
    feat.append(energy(acc_mag))         # Energy acc magnitude

    feat.append(np.mean(gyro_mag))       # Mean gyro magnitude
    feat.append(np.std(gyro_mag))        # Std gyro magnitude
    feat.append(energy(gyro_mag))         # Energy of magnitude

    features.append(feat)

# Convert list to NumPy array
processed_data = np.array(features)

# Separate features and labels
x = processed_data[:, 1:]  # Features: 6 axes × 9 stats + 6 global (acc/gyro mag) = 60 total
y = processed_data[:, 0]   # Labels (activity ID)

# Save processed data to file
np.savetxt("activity_new_data.txt", processed_data, delimiter=",", fmt="%.5f")

print("Data saved to 'activity_new_data.txt'")

#------------------------------------------------------------------------------------------------------------------
#   End of file
#--------------------------------------------

After all model evaluation... (models.ipynb and New_Models.ipynb)

In [ ]:
# ============================================
# Optimized and Calibrated Stacking Ensemble
# ============================================
import warnings
warnings.filterwarnings("ignore", message=".*use_label_encoder.*")
import numpy as np
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from scipy.stats import uniform, randint
from collections import Counter

# -----------------------
# 1. Load dataset
# -----------------------
data = np.loadtxt('activity_data_clean.txt')
X = data[:, 1:]
y = data[:, 0].astype(int)
print(Counter(y))

# -----------------------
# 2. Feature Selection
# -----------------------
feature_selector = SelectKBest(score_func=f_classif, k='all')  # keep all features

# -----------------------
# 3. Base Classifiers
# -----------------------
xgb = XGBClassifier(eval_metric='mlogloss', random_state=42)
gb = GradientBoostingClassifier(random_state=42)
et = ExtraTreesClassifier(class_weight='balanced', random_state=42)

# -----------------------
# 4. Tuning Space (XGBoost)
# -----------------------
param_dist_xgb = {
    'clf__n_estimators': randint(200, 400),
    'clf__learning_rate': uniform(0.01, 0.2),
    'clf__max_depth': randint(3, 8),
    'clf__subsample': uniform(0.6, 0.4),
    'clf__colsample_bytree': uniform(0.6, 0.4)
}

# -----------------------
# 5. Pipeline for Random Search
# -----------------------
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('select', feature_selector),
    ('clf', xgb)
])

# -----------------------
# 6. Randomized Search
# -----------------------
rskf = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist_xgb,
    n_iter=60,
    cv=rskf,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
random_search.fit(X, y)
best_xgb = random_search.best_estimator_

# -----------------------
# 7. Calibrate XGBoost
# -----------------------
calibrated_xgb = CalibratedClassifierCV(estimator=best_xgb, method='sigmoid', cv=5)
calibrated_xgb.fit(X, y)

# -----------------------
# 8. Stacking Ensemble
# -----------------------
stacking_model = StackingClassifier(
    estimators=[
        ('xgb', calibrated_xgb),
        ('gb', gb),
        ('et', et)
    ],
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

# -----------------------
# 9. Evaluate
# -----------------------
stacking_scores = cross_val_score(stacking_model, X, y, cv=5, scoring='accuracy')
print(f"\nStacking Ensemble Accuracy: {stacking_scores.mean():.4f} ± {stacking_scores.std():.4f}")
print("\nBest XGBoost parameters found:")
print(random_search.best_params_)

# Final Online Classification

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Online classification of mobile sensor data (accelerometer only, 30 best features embedded in training)
#------------------------------------------------------------------------------------------------------------------

import time
import requests
import numpy as np
import threading
from scipy.interpolate import interp1d
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from xgboost import XGBClassifier
from scipy import stats

##########################################
############ Data properties #############
##########################################

sampling_rate = 20
window_time = 0.5
window_samples = int(window_time * sampling_rate)

##########################################
##### Load data and train model here #####
##########################################

data = np.loadtxt("activity_new_data.txt", delimiter=",")
X = data[:, 1:]
y = data[:, 0].astype(int)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('select', SelectKBest(score_func=f_classif, k=30)),
    ('clf', XGBClassifier(
        colsample_bytree=0.8,
        learning_rate=0.05,
        max_depth=4,
        n_estimators=200,
        subsample=0.8,
        eval_metric='mlogloss',
    ))
])

pipeline.fit(X, y)  # ENTRENAMIENTO REAL con solo 30 columnas


##########################################
##### Data acquisition configuration #####
##########################################

IP_ADDRESS = '10.43.37.93'
COMMAND = 'accX&accY&accZ&gyroX&gyroY&gyroZ&acc_time'
BASE_URL = f"http://{IP_ADDRESS}/get?{COMMAND}"

max_samp_rate = 5000
n_signals = 6  
buffer_size = max_samp_rate * 5

buffer = np.zeros((buffer_size, n_signals + 1), dtype='float64')
buffer_index = 0
last_sample_time = 0.0

stop_recording_flag = threading.Event()
buffer_lock = threading.Lock()

def fetch_data():
    sleep_time = 1. / max_samp_rate
    while not stop_recording_flag.is_set():
        try:
            response = requests.get(BASE_URL, timeout=0.5)
            response.raise_for_status()
            data = response.json()

            global buffer, buffer_index, last_sample_time

            with buffer_lock:
                buffer[buffer_index, 0] = data["buffer"]["acc_time"]["buffer"][0]
                buffer[buffer_index, 1] = data["buffer"]["accX"]["buffer"][0]
                buffer[buffer_index, 2] = data["buffer"]["accY"]["buffer"][0]
                buffer[buffer_index, 3] = data["buffer"]["accZ"]["buffer"][0]
                buffer[buffer_index:, 4] = data["buffer"]["gyroX"]["buffer"][0]
                buffer[buffer_index:, 5] = data["buffer"]["gyroY"]["buffer"][0]
                buffer[buffer_index:, 6] = data["buffer"]["gyroZ"]["buffer"][0]

                buffer_index = (buffer_index + 1) % buffer_size
                last_sample_time = data["buffer"]["acc_time"]["buffer"][0]

        except Exception as e:
            print(f"Error fetching data: {e}")
        time.sleep(sleep_time)

def stop_recording():
    stop_recording_flag.set()
    recording_thread.join()

recording_thread = threading.Thread(target=fetch_data, daemon=True)
recording_thread.start()

##########################################
######### Online classification ##########
##########################################

def energy(sig):
    return np.sum(sig ** 2) / len(sig)

def zero_crossings(sig):
    return ((sig[:-1] * sig[1:]) < 0).sum()

activity_map = {
    0: "Walking",
    1: "Running",
    2: "Jumping Up and Down",
    3: "Greeting",
    4: "Sit Down",
    5: "Shadow Boxing"
}

update_time = 0.25
ref_time = time.time()

while True:
    time.sleep(update_time)

    if buffer_index > 2 * sampling_rate:
        ref_time = time.time()

        end_index = (buffer_index - 1) % buffer_size
        start_index = (buffer_index - 2) % buffer_size

        with buffer_lock:
            while (buffer[end_index, 0] - buffer[start_index, 0]) <= window_time:
                start_index = (start_index - 1) % buffer_size

            indices = (buffer_index - np.arange(buffer_size, 0, -1)) % buffer_size
            last_raw_data = buffer[indices, :]

        t = last_raw_data[:, 0]
        t_uniform = np.linspace(last_sample_time - window_time, last_sample_time, int(window_time * sampling_rate))

        last_data = np.zeros((len(t_uniform), n_signals))
        for i in range(n_signals):
            interp_x = interp1d(t, last_raw_data[:, i+1], kind='linear', fill_value="extrapolate")
            last_data[:, i] = interp_x(t_uniform)

        features = []
        for i in range(6):
            sig = last_data[:, i]
            features.extend([
                np.mean(sig),
                np.std(sig),
                np.max(sig),
                np.min(sig),
                stats.skew(sig),
                stats.kurtosis(sig),
                energy(sig),
                np.sqrt(np.mean(sig ** 2)),
                zero_crossings(sig)
            ])

        acc_mag = np.linalg.norm(last_data[:, 0:3], axis=1)
        gyro_mag = np.linalg.norm(last_data[:, 3:6], axis=1)
        features.extend([
        np.mean(acc_mag),
        np.std(acc_mag),
        energy(acc_mag),
        np.mean(gyro_mag),
        np.std(gyro_mag),
        energy(gyro_mag)
        ])



        X_new = np.array(features).reshape(1, -1)
        prediction = pipeline.predict(X_new)
        activity_label = activity_map.get(prediction[0], "Unknown")
        print(f"Predicción en tiempo real: {activity_label}")

stop_recording()

#------------------------------------------------------------------------------------------------------------------
#   End of file
#-------------------------------------------------